- Aggregate data into one Data Frame using Pandas. Pay attention that files may have different names for the same column. therefore, make sure that you unify the columns names before concating them.
- Standardizing header names
- Deleting and rearranging columns – delete the column customer as it is only a unique identifier for each row of data
- Working with data types – Check the data types of all the columns and fix the incorrect ones (for ex. customer lifetime value and number of open complaints ). Hint: remove the percentage from the customer lifetime value and truncate it to an integer value.
- clean the number of open complaints and extract the middle number which is changing between records. pay attention that the number of open complaints is a categorical feature.
- Filtering data and Correcting typos – Filter the data in state and gender column to standardize the texts in those columns
- Removing duplicates


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox

pd.options.display.max_rows = 100
## Install xlrd package to load Excel files
#!conda install openpyxl
#!conda install xlrd

In [2]:
def clean_columns():
    """
    clean column names;
    cast all as lowercase;
    replace "st" with "state";
    replace all spaces with "_"
    """
    f1 = pd.read_csv('Data/file1.csv')
    f2 = pd.read_csv('Data/file2.csv')
    f3 = pd.read_csv('Data/file3.csv')
    
    f1_cols = [i.lower().replace(" ", "_") for i in f1.columns]
    f1_cols[f1_cols.index("st")] = "state"
    
    f2_cols = [i.lower().replace(" ", "_") for i in f2.columns]
    f2_cols[f2_cols.index("st")] = "state"
    
    f3_cols = [i.lower().replace(" ", "_") for i in f3.columns]
    
    return f1_cols, f2_cols, f3_cols

cols_to_use = clean_columns()[0]

In [3]:
def load_original_data():
    """
    load original data with already sorted column order;
    concatenate data into one df
    """
    file1 = pd.read_csv('Data/file1.csv')
    file2 = pd.read_csv('Data/file2.csv')
    file3 = pd.read_csv('Data/file3.csv')
    
    file1_cols, file2_cols, file3_cols = clean_columns()
    
    file1.columns = file1_cols
    file2.columns = file2_cols
    file3.columns = file3_cols
    
    file1 = file1[cols_to_use]
    file2 = file2[cols_to_use]
    file3 = file3[cols_to_use]
    
    return pd.concat([file1,file2,file3], axis=0) # axis = 0 means row-wise; 1 means col-wise

ca_df = load_original_data()

In [4]:
def clean_gender_col():
    """
    clean gender column data;
    make everything either M, F, or NaN
    """
    gender_list = []
    
    for gender in ca_df["gender"]:
        if type(gender)==float:
            gender_list.append(np.nan)
        elif type(gender)!=float:
            if gender.lower().startswith("f"):
                gender_list.append("F")
            else:
                gender_list.append("M")

    ca_df["gender"] = gender_list
    
clean_gender_col()

In [15]:
def clean_states():

    state_list = []

    for state in ca_df["state"]:
        """
        func descr
        """
        if state.lower()=="az":
            state_list.append("Arizona")
        elif state.lower()=="wa":
            state_list.append("Washington")
        elif state.lower()=="cali":
            state_list.append("California")
        else:
            state_list.append(state)
    
    ca_df["state"] = state_list

clean_states()

In [5]:
def drop_column(col):
    """
    take a column and delete it from the dataframe
    """
    if col not in ca_df.columns:
        raise ValueError(f"Your given column '{col}' is not a column in the dataframe!")
        drop_column()
    
    ca_df.drop(col, axis=1, inplace=True)
    
    return ca_df

In [7]:
ca_df.dropna(how="all", inplace=True)
drop_column("customer")
ca_df["customer_lifetime_value"] = [int(entry[:entry.rfind(".")]) if type(entry)!=float
                                    else int(entry) if entry >= 0
                                    else np.nan for entry in ca_df["customer_lifetime_value"]]
ca_df["number_of_open_complaints"] = [int(entry[2:3]) if type(entry)==str else int(entry) for entry in ca_df["number_of_open_complaints"]]
ca_df.drop_duplicates(inplace=True)
ca_df.reset_index(drop=True, inplace=True)

hk_df['median_home_val'] =  pd.to_numeric(hk_df['median_home_val'], errors='coerce')

In [8]:
ca_df.isna().sum()

state                          0
gender                       122
education                      0
customer_lifetime_value        7
income                         0
monthly_premium_auto           0
number_of_open_complaints      0
policy_type                    0
vehicle_class                  0
total_claim_amount             0
dtype: int64

In [17]:
ca_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8879 entries, 0 to 8878
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   state                      8879 non-null   object 
 1   gender                     8757 non-null   object 
 2   education                  8879 non-null   object 
 3   customer_lifetime_value    8872 non-null   float64
 4   income                     8879 non-null   float64
 5   monthly_premium_auto       8879 non-null   float64
 6   number_of_open_complaints  8879 non-null   int64  
 7   policy_type                8879 non-null   object 
 8   vehicle_class              8879 non-null   object 
 9   total_claim_amount         8879 non-null   float64
dtypes: float64(4), int64(1), object(5)
memory usage: 693.8+ KB
